In [204]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

In [205]:
column_names = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'salary'
]

In [206]:
def strip_str(a_string):
    if a_string.dtype == 'string':
        return a_string.strip()
    
strip_udf = udf(strip_str, StringType())

In [207]:
#import pandas as pd
#train_df = pd.read_csv('work/adult.data', names=column_names)
#test_df = pd.read_csv('work/adult.test', names=column_names)[1:]

In [208]:
# train_df = train_df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

# train_df_cp = train_df.copy()

# train_df_cp = train_df_cp.loc[~train_df_cp['native-country'].isin(['Holand-Netherlands', '?'])]

# train_df_cp.to_csv('train.csv', index=False, header=False)

# test_df = test_df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

# test_df = test_df.loc[~test_df['native-country'].isin(['Holand-Netherlands', '?'])]

# test_df = test_df[test_df['native-country'].notna()]

# test_df.to_csv('test.csv', index=False, header=False)

In [209]:
spark = SparkSession.builder.appName("Predict Person Salary").getOrCreate()

In [215]:
schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("workclass", StringType(), True),
    StructField("fnlwgt", FloatType(), True),
    StructField("education", StringType(), True),
    StructField("education-num", FloatType(), True),
    StructField("marital-status", StringType(), True),
    StructField("occupation", StringType(), True),
    StructField("relationship", StringType(), True),
    StructField("race", StringType(), True),
    StructField("sex", StringType(), True),
    StructField("capital-gain", FloatType(), True),
    StructField("capital-loss", FloatType(), True),
    StructField("hours-per-week", FloatType(), True),
    StructField("native-country", StringType(), True),
    StructField("salary", StringType(), True)
])

In [216]:
train_df = spark.read.csv('work/adult.data', header=False, schema=schema)
test_df = spark.read.csv('work/adult.test', header=False, schema=schema)

In [247]:
train_df = train_df.filter(train_df['native-country'] != '	Holand-Netherlands')
test_df = test_df.filter(test_df['native-country'] != 'Holand-Netherlands')
# test_df = test_df.filter(test_df['workclass'].isNull)

In [254]:
train_df.select('native-country').distinct().toPandas()

,native-country
0,Dominican-Republic
1,Ireland
2,Cuba
3,Guatemala
4,Iran
5,Taiwan
6,El-Salvador
7,United-States
8,South
9,Japan


In [218]:
categorical_variables = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

indexers = [StringIndexer(inputCol=column, outputCol=column+"-index") for column in categorical_variables]

encoder = OneHotEncoder(
    inputCols=[indexer.getOutputCol() for indexer in indexers],
    outputCols=["{0}-encoded".format(indexer.getOutputCol()) for indexer in indexers]
)

assembler = VectorAssembler(
    inputCols=encoder.getOutputCols(),
    outputCol="categorical-features"
)

pipeline = Pipeline(stages=indexers + [encoder, assembler])
train_df = pipeline.fit(train_df).transform(train_df)
test_df = pipeline.fit(test_df).transform(test_df)

In [219]:
train_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: float (nullable = true)
 |-- education: string (nullable = true)
 |-- education-num: float (nullable = true)
 |-- marital-status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital-gain: float (nullable = true)
 |-- capital-loss: float (nullable = true)
 |-- hours-per-week: float (nullable = true)
 |-- native-country: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- workclass-index: double (nullable = false)
 |-- education-index: double (nullable = false)
 |-- marital-status-index: double (nullable = false)
 |-- occupation-index: double (nullable = false)
 |-- relationship-index: double (nullable = false)
 |-- race-index: double (nullable = false)
 |-- sex-index: double (nullable = false)
 |-- native-country-index: double (nullable 

In [220]:
train_df.limit(5).toPandas()['categorical-features'][0].toArray()

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [221]:
continuous_variables = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
assembler = VectorAssembler(
    inputCols=['categorical-features', *continuous_variables],
    outputCol='features'
)

In [222]:
train_df = assembler.transform(train_df)
test_df = assembler.transform(test_df)

In [223]:
train_df.limit(5).toPandas()['features'][0]

SparseVector(100, {4: 1.0, 10: 1.0, 24: 1.0, 32: 1.0, 44: 1.0, 48: 1.0, 52: 1.0, 53: 1.0, 94: 39.0, 95: 77516.0, 96: 13.0, 97: 2174.0, 99: 40.0})

In [224]:
indexer = StringIndexer(inputCol='salary', outputCol='label')
train_df = indexer.fit(train_df).transform(train_df)
test_df = indexer.fit(test_df).transform(test_df)
train_df.limit(10).toPandas()['label']

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
7    1.0
8    1.0
9    1.0
Name: label, dtype: float64

In [225]:
logireg = LogisticRegression(featuresCol='features', labelCol='label')
model = logireg.fit(train_df)

In [226]:
model.evaluate(test_df).accuracy

Py4JJavaError: An error occurred while calling o8787.accuracy.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1052.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1052.0 (TID 889) (abdca379ae24 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (ProbabilisticClassificationModel$$Lambda$4038/0x0000000801d44000: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:177)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.j

In [ ]:
pred = model.transform(test_df)
pred.limit(10).toPandas()[['label', 'prediction']]